In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta, timezone, date
from scipy.signal import find_peaks
import MetaTrader5 as mt5
from zoneinfo import ZoneInfo
import pytz
import time
from IPython.display import clear_output
from pandas import Timestamp
import sell_price_action_detector
import buy_price_action_detector

In [2]:
# Initialize MetaTrader 5 connection
if not mt5.initialize(login=89500879, server="MetaQuotes-Demo", password="6kF-FhYd"):
    print("Initialization failed")
    print(mt5.last_error())
    quit()

print("Connected to MT5 successfully")

Connected to MT5 successfully


In [3]:
#DROP THE UNNCESSARY COLUMNS IN THE DATAFRAME AND CONVERTING THE TIME TO HUMAN READABLE FORMAT
def df_convert_hr(rates_frame_input):
    rates_frame_input['time'] = pd.to_datetime(rates_frame_input['time'], unit='s')
    rates_frame_input.drop(["tick_volume", "spread", "real_volume"], inplace=True, axis=1)
    rates_frame_input["time"] = pd.to_datetime(rates_frame_input["time"])
    return rates_frame

def append_new_data(rates_frame, rates_frame_to_append):
    rates_frame = pd.concat([rates_frame, rates_frame_to_append]).drop_duplicates().reset_index(drop=True)
    return rates_frame

def download_recent_OHLCdata():
    global symbol
    timeframe = mt5.TIMEFRAME_M1
    
    #GETTING THE TIMESTAMP FOR TODAY AND YESTERDAY
    utc_from = datetime.now() - timedelta(minutes=3) + timedelta(hours=2)
    utc_to = datetime.now() + timedelta(hours=2) - timedelta(minutes=1)
    startdate=utc_from.strftime('%Y%m%d')
    enddate=utc_to.strftime('%Y%m%d')
    
    # Get the OHLC data FOR TODAY AND YESTERDAY
    rates_5m = mt5.copy_rates_range(symbol, timeframe, utc_from, utc_to)
    # Check if the data is downloaded successfully
    if rates is None:
        print("No data available, error code =", mt5.last_error())
    else:
        # Convert the data to a pandas DataFrame
        rates_frame_to_append = pd.DataFrame(rates_5m)
        df_convert_hr(rates_frame_to_append)
        return rates_frame_to_append

def download_and_display_updated_values():
    global rates_frame
    rates_frame_to_append = download_recent_OHLCdata()
    rates_frame = append_new_data(rates_frame, rates_frame_to_append)
    print("  ")
    print(f"{symbol} OHLC prices from the last 10 minutes :")
    print("____________________________________________________________________________")
    print(rates_frame.tail(10))
    return

In [4]:
#Detect the entry drills
def check_buy_entry_drill():
    if buy_tp == "-":
        Buy = False
        SL_price = None
        OB_size = None
        Entry_price = None
        SL_size = None
        p1 = p2 = p2_bos = p3 = p4 = buy_price_action_detector.create_empty_time_price_df()
        return Buy, SL_price, TP1_price, OB_size, Entry_price, SL_size, p1, p2, p2_bos, p3, p4
        
    global rates_frame, Last_110, rates_frame_p3, p3_rates_frame

    #check if pattern occured during pre-london session
    pre_ldn_session = 9 <= LowestPoint["time"].to_pydatetime().hour < 10
    #check if pattern occured during london session:
    ldn_session = 10 <= LowestPoint["time"].to_pydatetime().hour < 14
    #check if pattern occured during pre-ny session
    pre_ny_session = 14 <= LowestPoint["time"].to_pydatetime().hour < 15
    #check if pattern occured during ny session
    ny_session = 15 <= LowestPoint["time"].to_pydatetime().hour < 19
    
    condition1 = ldn_session and (ldn_low == LowestPoint["low"])
    condition2 = ny_session and (ny_low == LowestPoint["low"])
    condition3 = pre_ldn_session and (pre_ldn_low == LowestPoint["low"])
    condition4 = pre_ny_session and (pre_ny_low == LowestPoint["low"])
    is_OTE = condition1 or condition2 or condition3 or condition4
    
    if is_OTE:
        print('Price is at OTE of the session!')
        (
            Buy, SL_price, 
            TP1_price, OB_size,
            Entry_price, SL_size,
            p1, p2, p2_bos, p3, p4
        ) = buy_price_action_detector.validate_and_buy(rates_frame_p3, p3_rates_frame, buy_tp)
        return Buy, SL_price, TP1_price, OB_size, Entry_price, SL_size, p1, p2, p2_bos, p3, p4
    else:
        print("It is either price is not at the OTE of the session or it is not time to trade!")
        Buy = False
        SL_price = None
        TP1_price = None
        OB_size = None
        Entry_price = None
        SL_size = None
        p1 = p2 = p2_bos = p3 = p4 = buy_price_action_detector.create_empty_time_price_df()
        return Buy, SL_price, TP1_price, OB_size, Entry_price, SL_size, p1, p2, p2_bos, p3, p4

def check_sell_entry_drill():
    if sell_tp == "-":
        Sell = False
        SL_price = None
        TP1_price = None
        OB_size = None
        Entry_price = None
        SL_size = None
        p1 = p2 = p2_bos = p3 = p4 = sell_price_action_detector.create_empty_time_price_df()
        return Sell, SL_price, TP1_price, OB_size, Entry_price, SL_size, p1, p2, p2_bos, p3, p4
        
    global rates_frame, Last_110, rates_frame_p3, p3_rates_frame
    
    #check if pattern occured during pre-london session
    pre_ldn_session = 9 <= HighestPoint["time"].to_pydatetime().hour < 10
    #check if pattern occured during london session:
    ldn_session = 10 <= HighestPoint["time"].to_pydatetime().hour < 14
    #check if pattern occured during pre-ny session
    pre_ny_session = 14 <= HighestPoint["time"].to_pydatetime().hour < 15
    #check if pattern occured during ny session
    ny_session = 15 <= HighestPoint["time"].to_pydatetime().hour < 19

    condition1 = ldn_session and (ldn_high == HighestPoint["high"])
    condition2 = ny_session and (ny_high == HighestPoint["high"])
    condition3 = pre_ldn_session and (pre_ldn_high == HighestPoint["high"])
    condition4 = pre_ny_session and (pre_ny_high == HighestPoint["high"])
    is_OTE = condition1 or condition2 or condition3 or condition4
    
    if is_OTE:
        print('Price is at OTE of the session!')
        (
            Sell, SL_price, TP1_price, 
            OB_size, Entry_price, SL_size, 
            p1, p2, p2_bos, p3, p4 
        ) = sell_price_action_detector.validate_and_sell(rates_frame_p3, p3_rates_frame, sell_tp)
        return Sell, SL_price, TP1_price, OB_size, Entry_price, SL_size, p1, p2, p2_bos, p3, p4
    else:
        print("It is either price is not at the OTE of the session or it is not time to trade!")
        Sell = False
        SL_price = None
        TP1_price = None
        OB_size = None
        Entry_price = None
        SL_size = None
        p1 = p2 = p2_bos = p3 = p4 = sell_price_action_detector.create_empty_time_price_df()
        return Sell, SL_price, TP1_price, OB_size, Entry_price, SL_size, p1, p2, p2_bos, p3, p4

In [5]:
symbol = "BTC"
timeframe = mt5.TIMEFRAME_M1

#GETTING THE TIMESTAMP FOR TODAY AND YESTERDAY
utc_from = datetime.now() - timedelta(days=1) + timedelta(hours=2)
utc_to = datetime.now() - timedelta(minutes=1) + timedelta(hours=2)
startdate=utc_from.strftime('%Y%m%d')
enddate=utc_to.strftime('%Y%m%d')

# Get the OHLC data FOR TODAY AND YESTERDAY
rates = mt5.copy_rates_range(symbol, timeframe, utc_from, utc_to)
# Check if the data is downloaded successfully
if rates is None:
    print("No data available, error code =", mt5.last_error())
else:
    # Convert the data to a pandas DataFrame
    rates_frame = pd.DataFrame(rates)
    rates_frame = df_convert_hr(rates_frame)
    print(rates_frame)

                   time   open   high    low  close
0   2025-01-17 16:30:00  45.59  45.60  45.46  45.50
1   2025-01-17 16:31:00  45.40  45.50  45.40  45.50
2   2025-01-17 16:32:00  45.51  45.60  45.51  45.60
3   2025-01-17 16:33:00  45.64  45.72  45.60  45.65
4   2025-01-17 16:34:00  45.67  45.67  45.57  45.62
..                  ...    ...    ...    ...    ...
373 2025-01-17 22:55:00  46.52  46.53  46.52  46.53
374 2025-01-17 22:56:00  46.53  46.53  46.45  46.46
375 2025-01-17 22:57:00  46.45  46.45  46.40  46.40
376 2025-01-17 22:58:00  46.41  46.42  46.38  46.39
377 2025-01-17 22:59:00  46.43  46.51  46.43  46.47

[378 rows x 5 columns]


In [6]:
try:
    current_price = float(rates_frame["close"].tail(1).iloc[0])
    print(f"The current price of {symbol} is about: {current_price}.")

    while True:
        sell_tp = input("Enter the sell TP: ").strip()
        try: 
            # Check for a dash
            if sell_tp == "-":
                print("You are only looking to buy.")
                BEARISH_BIAS = False
                break
        
            sell_tp = float(sell_tp)
            if sell_tp >= current_price:
                print(f"Enter a price that is less than the current price of {symbol}.")
                continue
            value = float(sell_tp)
            print(f"You entered a TP level: {value}")
            BEARISH_BIAS = True
            break
        except ValueError:
            print(f"Invalid input. Please enter a dash '-' or a valid price of {symbol}.")
            continue
    
    while True:
        buy_tp = input("Enter the buy TP: ").strip()
        try:
            if buy_tp == "-":
                print("You are only looking to sell.")
                BULLISH_BIAS = False
                break
            
            buy_tp = float(buy_tp) 
            if buy_tp <= current_price:
                print(f"Enter a price that is more than the current price of {symbol}.")
                continue
                
            value = float(buy_tp)
            print(f"You entered a TP level: {value}")
            BULLISH_BIAS = True
            break
            
        except ValueError:
            print(f"Invalid input. Please enter a dash '-' or a valid price of {symbol}.")
            continue
            
except IndexError as err1:
    print(f'IndexError : {err1} ')
except NameError as err2:
    print(f'Name error 1 : {err2} ')

The current price of BTC is about: 46.47.


Enter the sell TP:  0


You entered a TP level: 0.0


Enter the buy TP:  -


You are only looking to sell.


In [ ]:
#number of trades for today
N = 0
date_today = date.today()

while N < 3:
    if N >= 3:
        break
    else:
        clear_output(wait=True)
        #HAVE THE DATAFRAME FOR THE CURRENT LONDON SESSION and LOOK FOR THE LONDON HIGH AND LOW:
        pre_ldn_rates_frame = rates_frame[(rates_frame['time'].dt.date == date_today) & (rates_frame['time'].dt.hour >= 9) & (rates_frame['time'].dt.hour < 10)]
        pre_ldn_high = pre_ldn_rates_frame["high"].max()
        pre_ldn_low = pre_ldn_rates_frame["low"].min()
        print("__________________________________")
        print(f'Pre-London high: {pre_ldn_high}')
        print(f'Pre-London low:  {pre_ldn_low}\n')
        
        ldn_rates_frame = rates_frame[(rates_frame['time'].dt.date == date_today) & (rates_frame['time'].dt.hour >= 10) & (rates_frame['time'].dt.hour < 14)]
        ldn_high = ldn_rates_frame["high"].max()
        ldn_low = ldn_rates_frame["low"].min()
        print(f'London high:.....{ldn_high}')
        print(f'London low:......{ldn_low}\n')

        
        #HAVE THE DATAFRAME FOR THE CURRENT NEW YORK SESSION and LOOK FOR THE NY HIGH AND LOW: 
        pre_ny_rates_frame = rates_frame[(rates_frame['time'].dt.date == date_today) & (rates_frame['time'].dt.hour >= 14) & (rates_frame['time'].dt.hour < 15)]
        pre_ny_high = pre_ny_rates_frame["high"].max()
        pre_ny_low = pre_ny_rates_frame["low"].min()
        print(f'Pre-NY high:.....{pre_ny_high}')
        print(f'Pre-NY low:......{pre_ny_low}\n')

        ny_rates_frame = rates_frame[(rates_frame['time'].dt.date == date_today) & (rates_frame['time'].dt.hour >= 15) & (rates_frame['time'].dt.hour < 19)]
        ny_high = ny_rates_frame["high"].max()
        ny_low = ny_rates_frame["low"].min()
        print(f'NY high:.........{ny_high}')
        print(f'NY low:..........{ny_low}')
        print("__________________________________")
        
        try:
            #Determine the max and min value of the highs and lows for the last 35 mins
            Last_110 = rates_frame.tail(110)
            MaxHighPrice = Last_110["high"].max()
            MinLowPrice = Last_110["low"].min()
    
            #Determine the if the last 80 mins is in a downtrend or uptrend
            HighestPoint = rates_frame[rates_frame["high"] == MaxHighPrice].iloc[-1]
            LowestPoint = rates_frame[rates_frame["low"] == MinLowPrice].iloc[-1]
        except IndexError as err1:
            print("It is weekend you dumbass! The forex market is closed!")
            
        #Based on the trend (and the conditions inside) decide whether to buy, sell, or wait.
        try:
            print(f'The time today is: {datetime.now().strftime("%I:%M %p")}')
            if (HighestPoint["time"] > LowestPoint["time"]) & BEARISH_BIAS:
                print("Looking to sell...")
                MaxHighPrice = Last_110["high"].max()
                MaxHighPoint = Last_110[Last_110["high"] == MaxHighPrice].iloc[[0]]
                MaxHighTime = MaxHighPoint['time'].iloc[0]
                rates_frame_p3 = rates_frame[rates_frame["time"]<= MaxHighTime].tail(110)
                p3_rates_frame = rates_frame[rates_frame["time"]>= MaxHighTime]
                Sell, SL_price, TP1_price, OB_size, Entry_price, SL_size, p1, p2, p2_bos, p3, p4 = check_sell_entry_drill()
                print(f'p1: {p1}\n\np2: {p2}\n\np3: {p3}')
                print(f'Sell: {Sell}')
                download_and_display_updated_values()
                time.sleep(60)
                continue
            elif (HighestPoint["time"] < LowestPoint["time"]) & BULLISH_BIAS:
                print("Looking to buy...")
                MinLowPrice = Last_110["low"].min()
                MinLowPoint = Last_110[Last_110["low"] == MinLowPrice].iloc[[0]]
                MinLowTime = MinLowPoint['time'].iloc[0]
                rates_frame_p3 = rates_frame[rates_frame["time"]<= MinLowTime].tail(110)
                p3_rates_frame = rates_frame[rates_frame["time"]>= MinLowTime]                
                Buy, SL_price, TP1_price, OB_size, Entry_price, SL_size, p1, p2, p2_bos, p3, p4 = check_buy_entry_drill()
                print(f'p1: {p1}\n\np2: {p2}\n\np3: {p3}')
                print(f'Buy: {Buy}')
                download_and_display_updated_values()
                time.sleep(60)
                continue
            else:
                if BULLISH_BIAS and BEARISH_BIAS:
                    print("Waiting for an opportunity for both buy and sell...")
                elif BULLISH_BIAS:
                    print("Waiting for an opportunity to buy...")
                elif BEARISH_BIAS:
                    print("Waiting for an opportunity to sell...")
                download_and_display_updated_values()
                time.sleep(60)
                continue
        except NameError as err1:
            print(f'Name error 2 : {err1} ')
            break

__________________________________
Pre-London high: nan
Pre-London low:  nan

London high:.....nan
London low:......nan

Pre-NY high:.....nan
Pre-NY low:......nan

NY high:.........nan
NY low:..........nan
__________________________________
The time today is: 07:36 PM
Waiting for an opportunity to sell...
  
BTC OHLC prices from the last 10 minutes :
____________________________________________________________________________
                   time   open   high    low  close
368 2025-01-17 22:50:00  46.56  46.57  46.56  46.57
369 2025-01-17 22:51:00  46.56  46.56  46.51  46.51
370 2025-01-17 22:52:00  46.54  46.54  46.52  46.53
371 2025-01-17 22:53:00  46.56  46.56  46.55  46.55
372 2025-01-17 22:54:00  46.57  46.59  46.57  46.58
373 2025-01-17 22:55:00  46.52  46.53  46.52  46.53
374 2025-01-17 22:56:00  46.53  46.53  46.45  46.46
375 2025-01-17 22:57:00  46.45  46.45  46.40  46.40
376 2025-01-17 22:58:00  46.41  46.42  46.38  46.39
377 2025-01-17 22:59:00  46.43  46.51  46.43  46.4